In [ ]:
import pandas as pd
from prophet.plot import plot_plotly, plot_components_plotly
from prophet import Prophet
from statsmodels.tools.eval_measures import rmse

df = pd.read_csv('/Users/alanchan/Documents/UCL/Year 4/ELEC0088/SNS assignment 22-23/London Property Price/ukhpi-property-type-london-from-1977-06-01-to-2018-06-01.csv')
print(df.info())
df.dropna(inplace= True)
df.reset_index(drop=True, inplace=True)
print(df.info())

In [ ]:
# print(df.tail())
df=df[["Period","Average price All property types"]]
df.columns = ['ds','y']
df['ds'] = pd.to_datetime(df['ds'], format = '%Y-%m')
print(df.tail())

In [ ]:
df.sort_values(by='ds', inplace = True)
df.reset_index(drop=True, inplace=True)
# print(df.head())
print(df.tail())

In [ ]:
df.plot(x='ds',y='y',figsize=(18,6))
print(len(df))

In [ ]:
Test_split = 12
train = df.iloc[:len(df)-Test_split]
test = df.iloc[len(df)-Test_split:]

print(len(train))
print(len(test))

print(df.tail())

In [ ]:
m = Prophet()
m.fit(train)
future = m.make_future_dataframe(periods=12, freq = 'MS')
forecast = m.predict(future)
print(df.tail())
print(future.tail())

# forecast.tail()
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
# future.tail()

In [ ]:
plot_plotly(m ,forecast)

In [ ]:
plot_components_plotly(m, forecast)

In [ ]:
predictions = forecast.iloc[len(train):len(train)+len(test)]['yhat']
print("Root Mean Squared Error between actual and  predicted values: ",rmse(predictions,test['y']))
print("Mean Value of Test Dataset:", test['y'].mean())

In [ ]:
training = forecast.iloc[:len(train)]['yhat']
print("Root Mean Squared Error between actual and  predicted values: ",rmse(training,train['y']))